In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd /content/gdrive/MyDrive

/content/gdrive/My Drive


#**Download MARBERT checkpoint**

In [ ]:
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz

--2023-05-25 18:56:34--  https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
Resolving huggingface.co (huggingface.co)... 18.172.170.14, 18.172.170.44, 18.172.170.36, ...
Connecting to huggingface.co (huggingface.co)|18.172.170.14|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27MARBERT_pytorch_verison.tar.gz%3B+filename%3D%22MARBERT_pytorch_verison.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1685300195&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL1VCQy1OTFAvTUFSQkVSVC84NWJmZWM3NmYzOGNiYTRiYzJlNmNkMDJhOTU5MDE2ZGUzN2JhOTNkZTRjODUwYTdkMTc1ODExZGNlNGU4YWRjP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4NTMwM

In [ ]:
!tar -xvf MARBERT_pytorch_verison.tar.gz

MARBERT_pytorch_verison/
MARBERT_pytorch_verison/pytorch_model.bin
MARBERT_pytorch_verison/config.json
MARBERT_pytorch_verison/vocab.txt


In [ ]:
import os
import pandas as pd


In [ ]:
!pip install GPUtil pytorch_pretrained_bert transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.0 MB/s eta 0:00:00
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=60a575d73d568dec1c3f5d2f711a4319bbd087a78da5b3a3cc0c00c74dbeebdd
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


# Fine-tuning code

In [ ]:
# (1)load libraries 
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [ ]:

def create_label2ind_file(file, label_col):
	labels_json={}
	#load train_dev_test file
	df = pd.read_csv(file, sep="\t")
	df.head(5)
	#get labels and sort it A-Z
	labels = df[label_col].unique()
	labels.sort()
	#convert labels to indexes
	for idx in range(0, len(labels)):
		labels_json[labels[idx]]=idx
	print(labels_json)
	#save labels with indexes to file
	with open(label2idx_file, 'w') as json_file:
		json.dump(labels_json, json_file)


In [ ]:
def data_prepare_BERT(file_path, lab2ind, tokenizer, content_col, label_col, MAX_LEN):
	# Use pandas to load dataset
	df = pd.read_csv(file_path, delimiter='\t', header=0)
	df = df[df[content_col].notnull()]
	df = df[df[label_col].notnull()]
	print("Data size ", df.shape)
	# Create sentence and label lists
	sentences = df[content_col].values
	sentences = ["[CLS] " + sentence+ " [SEP]" for sentence in sentences]
	print ("The first sentence:")
	print (sentences[0])
	# Create sentence and label lists
	labels = df[label_col].values
	#print (labels)
	labels = [lab2ind[i] for i in labels]
	# Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
	tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
	print ("Tokenize the first sentence:")
	print (tokenized_texts[0])
	#print("Label is ", labels[0])
	# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
	input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
	print ("Index numbers of the first sentence:")
	print (input_ids[0])
	# Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
	# ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
	pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
	input_ids = pad_sequences(input_ids, maxlen=MAX_LEN+2, dtype="long", truncating="post", padding="post", value=pad_ind)
	print ("Index numbers of the first sentence after padding:\n",input_ids[0])
	# Create attention masks
	attention_masks = []
	# Create a mask of 1s for each token followed by 0s for padding
	for seq in input_ids:
		seq_mask = [float(i > 0) for i in seq]
		attention_masks.append(seq_mask)
	# Convert all of our data into torch tensors, the required datatype for our model
	inputs = torch.tensor(input_ids)
	labels = torch.tensor(labels)
	masks = torch.tensor(attention_masks)
	return inputs, labels, masks


In [ ]:
# Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#	  pred_flat = np.argmax(preds, axis=1).flatten()
#	  labels_flat = labels.flatten()
#	  return np.sum(pred_flat == labels_flat) / len(labels_flat)
def flat_pred(preds, labels):
	pred_flat = np.argmax(preds, axis=1).flatten()
	labels_flat = labels.flatten()
	return pred_flat.tolist(), labels_flat.tolist()

In [ ]:

def train(model, iterator, optimizer, scheduler, criterion):
	
	model.train()
	epoch_loss = 0
	for i, batch in enumerate(iterator):
		# Add batch to GPU
		batch = tuple(t.to(device) for t in batch)
		# Unpack the inputs from our dataloader
		input_ids, input_mask, labels = batch
		outputs = model(input_ids, input_mask, labels=labels)
		loss, logits = outputs[:2]
		#print("loss ",loss)
		# delete used variables to free GPU memory
		del batch, input_ids, input_mask, labels
		optimizer.zero_grad()
		if torch.cuda.device_count() == 1:
			loss.backward()
			epoch_loss += loss.cpu().item()
		else:
			loss.sum().backward()
			epoch_loss += loss.sum().cpu().item()
		optimizer.step()
		#print("optimizer step")
		torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
		# optimizer.step()
		scheduler.step()
	# free GPU memory
	if device == 'cuda':
		torch.cuda.empty_cache()
	print("will return now")
	return epoch_loss / len(iterator)

In [ ]:

def evaluate(model, iterator, criterion):
	model.eval()
	epoch_loss = 0
	all_pred=[]
	all_label = []
	with torch.no_grad():
		for i, batch in enumerate(iterator):
			# Add batch to GPU
			batch = tuple(t.to(device) for t in batch)
			# Unpack the inputs from our dataloader
			input_ids, input_mask, labels = batch
			outputs = model(input_ids, input_mask, labels=labels)
			loss, logits = outputs[:2]
			# delete used variables to free GPU memory
			del batch, input_ids, input_mask
			if torch.cuda.device_count() == 1:
				epoch_loss += loss.cpu().item()
			else:
				epoch_loss += loss.sum().cpu().item()
			# identify the predicted class for each example in the batch
			probabilities, predicted = torch.max(logits.cpu().data, 1)
			# put all the true labels and predictions to two lists
			all_pred.extend(predicted)
			all_label.extend(labels.cpu())
	accuracy = accuracy_score(all_label, all_pred)
	f1score = f1_score(all_label, all_pred, average='macro') 
	recall = recall_score(all_label, all_pred, average='macro')
	precision = precision_score(all_label, all_pred, average='macro')
	report = classification_report(all_label, all_pred)
	return (epoch_loss / len(iterator)), accuracy, f1score, recall, precision



In [ ]:

def fine_tuning(config):
	#---------------------------------------
	print ("[INFO] step (1) load train_test config file")
	# config_file = open(config_file, 'r', encoding="utf8")
	# config = json.load(config_file)
	task_name = config["task_name"]
	content_col = config["content_col"]
	label_col = config["label_col"]
	train_file = config["data_dir"]+config["train_file"]
	dev_file = config["data_dir"]+config["dev_file"]
	sortby = config["sortby"]
	max_seq_length= int(config["max_seq_length"])
	batch_size = int(config["batch_size"])
	lr_var = float(config["lr"])
	model_path = config['pretrained_model_path']
	num_epochs = config['epochs'] # Number of training epochs (authors recommend between 2 and 4)
	global label2idx_file
	label2idx_file = config["data_dir"]+config["task_name"]+"_labels-dict.json"
	#-------------------------------------------------------
	print ("[INFO] step (2) convert labels2index")
	create_label2ind_file(train_file, label_col)
	print (label2idx_file)
	#---------------------------------------------------------
	print ("[INFO] step (3) check checkpoint directory and report file")
	ckpt_dir = config["data_dir"]+task_name+"_bert_ckpt/"
	report = ckpt_dir+task_name+"_report.tsv"
	sorted_report = ckpt_dir+task_name+"_report_sorted.tsv"
	if not os.path.exists(ckpt_dir):
		os.mkdir(ckpt_dir)
	#-------------------------------------------------------
	print ("[INFO] step (4) load label to number dictionary")
	lab2ind = json.load(open(label2idx_file))
	print ("[INFO] train_file", train_file)
	print ("[INFO] dev_file", dev_file)
	print ("[INFO] num_epochs", num_epochs)
	print ("[INFO] model_path", model_path)
	print ("max_seq_length", max_seq_length, "batch_size", batch_size)
	#-------------------------------------------------------
	print ("[INFO] step (5) Use defined funtion to extract tokanize data")
	# tokenizer from pre-trained BERT model
	print ("loading BERT setting")
	tokenizer = BertTokenizer.from_pretrained(model_path)
	train_inputs, train_labels, train_masks = data_prepare_BERT(train_file, lab2ind, tokenizer,content_col, label_col, max_seq_length)
	validation_inputs, validation_labels, validation_masks = data_prepare_BERT(dev_file, lab2ind, tokenizer, content_col, label_col,max_seq_length)
	# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
	model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(lab2ind))
	#--------------------------------------
	print ("[INFO] step (6) Create an iterator of data with torch DataLoader.")
#		  This helps save on memory during training because, unlike a for loop,\
#		  with an iterator the entire dataset does not need to be loaded into memory")
	train_data = TensorDataset(train_inputs, train_masks, train_labels)
	train_dataloader = DataLoader(train_data, batch_size=batch_size)
	#---------------------------
	validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
	validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
	#------------------------------------------
	print ("[INFO] step (7) run with parallel GPUs")
	if torch.cuda.is_available():
		if torch.cuda.device_count() == 1:
			print("Run", "with one GPU")
			model = model.to(device)
		else:
			n_gpu = torch.cuda.device_count()
			print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
			device_ids = GPUtil.getAvailable(limit = 4)
			torch.backends.cudnn.benchmark = True
			model = model.to(device)
			model = nn.DataParallel(model, device_ids=device_ids)
	else:
		print("Run", "with CPU")
		model = model
	#---------------------------------------------------
	print ("[INFO] step (8) set Parameters, schedules, and loss function")
	global max_grad_norm
	max_grad_norm = 1.0
	warmup_proportion = 0.1
	num_training_steps	= len(train_dataloader) * num_epochs
	num_warmup_steps = num_training_steps * warmup_proportion
	### In Transformers, optimizer and schedules are instantiated like this:
	# Note: AdamW is a class from the huggingface library
	# the 'W' stands for 'Weight Decay"
	optimizer = AdamW(model.parameters(), lr=lr_var, correct_bias=False)
	# schedules
	scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler
	# We use nn.CrossEntropyLoss() as our loss function. 
	criterion = nn.CrossEntropyLoss()
	#---------------------------------------------------
	print ("[INFO] step (9) start fine_tuning")
	for epoch in trange(num_epochs, desc="Epoch"):
		train_loss = train(model, train_dataloader, optimizer, scheduler, criterion)	  
		val_loss, val_acc, val_f1, val_recall, val_precision = evaluate(model, validation_dataloader, criterion)
		print (train_loss, val_acc)
		# Create checkpoint at end of each epoch
		if not os.path.exists(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/'): os.mkdir(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/')
		model.save_pretrained(ckpt_dir+ 'model_' + str(int(epoch + 1)) + '/')
		epoch_eval_results = {"epoch_num":int(epoch + 1),"train_loss":train_loss,
					  "val_acc":val_acc, "val_recall":val_recall, "val_precision":val_precision, "val_f1":val_f1,"lr":lr_var }
		with open(report,"a") as fOut:
			fOut.write(json.dumps(epoch_eval_results)+"\n")
			fOut.flush()
		#------------------------------------
		report_df = pd.read_json(report, orient='records', lines=True)
		report_df.sort_values(by=[sortby],ascending=False, inplace=True)
		report_df.to_csv(sorted_report,sep="\t",index=False)
	return report_df

# **Control Experiment for 1 Epoch**

In [ ]:

config={"task_name": "AJGT_MARBERT", #output directory name
             "data_dir": "./", #data directory
             "train_file": "ar_rev_train.tsv", #train file path
             "dev_file": "ar_rev_test.tsv", #dev file path or test file path
             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
             "epochs": 1, #number of epochs
             "content_col": "text", #text column
             "label_col": "label", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 128, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
report_df = fine_tuning(config)

[INFO] step (1) load train_test config file
[INFO] step (2) convert labels2index


loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file MARBERT_pytorch_verison/config.json


{'Mixed': 0, 'Negative': 1, 'Positive': 2}
./AJGT_MARBERT_labels-dict.json
[INFO] step (3) check checkpoint directory and report file
[INFO] step (4) load label to number dictionary
[INFO] train_file ./ar_rev_train.tsv
[INFO] dev_file ./ar_rev_test.tsv
[INFO] num_epochs 1
[INFO] model_path MARBERT_pytorch_verison
max_seq_length 128 batch_size 32
[INFO] step (5) Use defined funtion to extract tokanize data
loading BERT setting
Data size  (79999, 2)
The first sentence:
[CLS] رواية بسيطه عن مغترب تونسي يزور عائلته في اجازه قصيره ويلمس تغير البلاد والافكار الدينيه التي غزت العقول وحجم التناقض الكبير بين القول والفعل ..رواية جميله على عكس روايته السابقه روائح ماري كلير .. [SEP]
Tokenize the first sentence:
['[CLS]', 'رواية', 'بسيطه', 'عن', 'مغترب', 'تونسي', 'يزور', 'عايلته', 'في', 'اجازه', 'قصيره', 'ويلم', '##س', 'تغير', 'البلاد', 'والافكار', 'الدينيه', 'التي', 'غز', '##ت', 'العقول', 'وحجم', 'التناقض', 'الكبير', 'بين', 'القول', 'والفعل', '.', '.', 'رواية', 'جميله', 'على', 'عكس', 'رواي', '##

loading configuration file MARBERT_pytorch_verison/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file MARBERT_pytorch_verison/pytorch_

[INFO] step (6) Create an iterator of data with torch DataLoader.
[INFO] step (7) run with parallel GPUs
Run with one GPU
[INFO] step (8) set Parameters, schedules, and loss function
[INFO] step (9) start fine_tuning


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

will return now


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_1/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_1/pytorch_model.bin
Epoch: 100%|██████████| 1/1 [31:56<00:00, 1916.04s/it]


In [ ]:
report_df.head(5)

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
0,1,0.668333,0.727,0.72752,0.72458,0.724107,0.000002


# **Control Experiment for 5 Epochs**

In [ ]:

config_5={"task_name": "AJGT_MARBERT", #output directory name
             "data_dir": "./", #data directory
             "train_file": "ar_rev_train.tsv", #train file path
             "dev_file": "ar_rev_test.tsv", #dev file path or test file path
             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
             "epochs": 5, #number of epochs
             "content_col": "text", #text column
             "label_col": "label", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 128, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
report_df = fine_tuning(config_5)

[INFO] step (1) load train_test config file
[INFO] step (2) convert labels2index


loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file MARBERT_pytorch_verison/config.json


{'Mixed': 0, 'Negative': 1, 'Positive': 2}
./AJGT_MARBERT_labels-dict.json
[INFO] step (3) check checkpoint directory and report file
[INFO] step (4) load label to number dictionary
[INFO] train_file ./ar_rev_train.tsv
[INFO] dev_file ./ar_rev_test.tsv
[INFO] num_epochs 5
[INFO] model_path MARBERT_pytorch_verison
max_seq_length 128 batch_size 32
[INFO] step (5) Use defined funtion to extract tokanize data
loading BERT setting
Data size  (79999, 2)
The first sentence:
[CLS] رواية بسيطه عن مغترب تونسي يزور عائلته في اجازه قصيره ويلمس تغير البلاد والافكار الدينيه التي غزت العقول وحجم التناقض الكبير بين القول والفعل ..رواية جميله على عكس روايته السابقه روائح ماري كلير .. [SEP]
Tokenize the first sentence:
['[CLS]', 'رواية', 'بسيطه', 'عن', 'مغترب', 'تونسي', 'يزور', 'عايلته', 'في', 'اجازه', 'قصيره', 'ويلم', '##س', 'تغير', 'البلاد', 'والافكار', 'الدينيه', 'التي', 'غز', '##ت', 'العقول', 'وحجم', 'التناقض', 'الكبير', 'بين', 'القول', 'والفعل', '.', '.', 'رواية', 'جميله', 'على', 'عكس', 'رواي', '##

loading configuration file MARBERT_pytorch_verison/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file MARBERT_pytorch_verison/pytorch_

[INFO] step (6) Create an iterator of data with torch DataLoader.
[INFO] step (7) run with parallel GPUs
Run with one GPU
[INFO] step (8) set Parameters, schedules, and loss function
[INFO] step (9) start fine_tuning


...

Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_3/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_3/pytorch_model.bin
Epoch:  60%|██████    | 3/5 [1:31:50<1:01:14, 1837.39s/it]

will return now


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_4/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_4/pytorch_model.bin
Epoch:  80%|████████  | 4/5 [2:02:19<30:34, 1834.19s/it]  

will return now


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_5/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_5/pytorch_model.bin
Epoch: 100%|██████████| 5/5 [2:32:47<00:00, 1833.46s/it]


In [ ]:
print(report_df)

   epoch_num  train_loss  val_acc  val_recall  val_precision    val_f1  \
0        1.0    0.726922  0.73035    0.730780       0.729674  0.730171   
1        2.0    0.579407  0.73640    0.736757       0.737491  0.737082   
2        3.0    0.539269  0.73830    0.738669       0.738982  0.738793   
3        4.0    0.510306  0.73945    0.739751       0.740842  0.739712   
4        5.0    0.492171  0.73860    0.739036       0.737268  0.737666   

         lr  
0  0.000002  
1  0.000002  
2  0.000002  
3  0.000002  
4  0.000002  


# **Spell Checked Experiment for 5 Epochs**

splitting data to train and test

In [ ]:
df = pd.read_csv("spellchecked.tsv", sep='\t')
df_text_label = df[['cleaned_no_sw', 'label']]
print(df_text_label)
train, test = train_test_split(df_text_label, test_size=0.2, random_state=42, shuffle=True)
print(test)
train.to_csv("spell_checked_train.tsv", sep='\t', index=False)
test.to_csv("spell_checked_test.tsv", sep='\t', index=False)

In [ ]:

config_spellchecked={"task_name": "AJGT_MARBERT", #output directory name
             "data_dir": "./", #data directory
             "train_file": "spell_checked_train.tsv", #train file path
             "dev_file": "spell_checked_test.tsv", #dev file path or test file path
             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
             "epochs": 5, #number of epochs
             "content_col": "text", #text column
             "label_col": "label", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 128, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
report_df = fine_tuning(config_spellchecked)

[INFO] step (1) load train_test config file
[INFO] step (2) convert labels2index
{'Mixed': 0, 'Negative': 1, 'Positive': 2}


loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file MARBERT_pytorch_verison/config.json


./AJGT_MARBERT_labels-dict.json
[INFO] step (3) check checkpoint directory and report file
[INFO] step (4) load label to number dictionary
[INFO] train_file ./spell_checked_train.tsv
[INFO] dev_file ./spell_checked_test.tsv
[INFO] num_epochs 5
[INFO] model_path MARBERT_pytorch_verison
max_seq_length 128 batch_size 32
[INFO] step (5) Use defined funtion to extract tokanize data
loading BERT setting
Data size  (79999, 2)
The first sentence:
[CLS] رواية بسيطه عن مغترب تونسي يزور عائلته في اجازه قصيره ويلمس تغير البلاد والافكار الدينيه التي غزت العقول وحجم التناقض الكبير بين القول والفعل ..رواية جميله على عكس روايته السابقه روائح ماري كلير .. [SEP]
Tokenize the first sentence:
['[CLS]', 'رواية', 'بسيطه', 'عن', 'مغترب', 'تونسي', 'يزور', 'عايلته', 'في', 'اجازه', 'قصيره', 'ويلم', '##س', 'تغير', 'البلاد', 'والافكار', 'الدينيه', 'التي', 'غز', '##ت', 'العقول', 'وحجم', 'التناقض', 'الكبير', 'بين', 'القول', 'والفعل', '.', '.', 'رواية', 'جميله', 'على', 'عكس', 'رواي', '##ته', 'السابقه', 'روايح', 'مار

loading configuration file MARBERT_pytorch_verison/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file MARBERT_pytorch_verison/pytorch_

[INFO] step (6) Create an iterator of data with torch DataLoader.
[INFO] step (7) run with parallel GPUs
Run with one GPU
[INFO] step (8) set Parameters, schedules, and loss function
[INFO] step (9) start fine_tuning


...

Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_3/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_3/pytorch_model.bin
Epoch:  60%|██████    | 3/5 [1:35:23<1:03:35, 1907.82s/it]

will return now


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_4/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_4/pytorch_model.bin
Epoch:  80%|████████  | 4/5 [2:07:08<31:46, 1906.51s/it]  

will return now


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_5/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_5/pytorch_model.bin
Epoch: 100%|██████████| 5/5 [2:38:57<00:00, 1907.47s/it]


In [ ]:
print(report_df)

   epoch_num  train_loss  val_acc  val_recall  val_precision    val_f1  \
0        1.0    0.722939  0.73055    0.730935       0.730665  0.730780   
1        2.0    0.578857  0.73750    0.737840       0.738650  0.738193   
2        3.0    0.539652  0.73760    0.737885       0.739714  0.738508   
3        4.0    0.510117  0.73645    0.736740       0.738302  0.736791   
4        5.0    0.490089  0.73535    0.735781       0.734194  0.734503   

         lr  
0  0.000002  
1  0.000002  
2  0.000002  
3  0.000002  
4  0.000002  


# **MLSML Loss Function Experiment for 2 Epochs**

In [ ]:

def fine_tuning_MLSML(config):
	#---------------------------------------
	print ("[INFO] step (1) load train_test config file")
	# config_file = open(config_file, 'r', encoding="utf8")
	# config = json.load(config_file)
	task_name = config["task_name"]
	content_col = config["content_col"]
	label_col = config["label_col"]
	train_file = config["data_dir"]+config["train_file"]
	dev_file = config["data_dir"]+config["dev_file"]
	sortby = config["sortby"]
	max_seq_length= int(config["max_seq_length"])
	batch_size = int(config["batch_size"])
	lr_var = float(config["lr"])
	model_path = config['pretrained_model_path']
	num_epochs = config['epochs'] # Number of training epochs (authors recommend between 2 and 4)
	global label2idx_file
	label2idx_file = config["data_dir"]+config["task_name"]+"_labels-dict.json"
	#-------------------------------------------------------
	print ("[INFO] step (2) convert labels2index")
	create_label2ind_file(train_file, label_col)
	print (label2idx_file)
	#---------------------------------------------------------
	print ("[INFO] step (3) check checkpoint directory and report file")
	ckpt_dir = config["data_dir"]+task_name+"_bert_ckpt/"
	report = ckpt_dir+task_name+"_report.tsv"
	sorted_report = ckpt_dir+task_name+"_report_sorted.tsv"
	if not os.path.exists(ckpt_dir):
		os.mkdir(ckpt_dir)
	#-------------------------------------------------------
	print ("[INFO] step (4) load label to number dictionary")
	lab2ind = json.load(open(label2idx_file))
	print ("[INFO] train_file", train_file)
	print ("[INFO] dev_file", dev_file)
	print ("[INFO] num_epochs", num_epochs)
	print ("[INFO] model_path", model_path)
	print ("max_seq_length", max_seq_length, "batch_size", batch_size)
	#-------------------------------------------------------
	print ("[INFO] step (5) Use defined funtion to extract tokanize data")
	# tokenizer from pre-trained BERT model
	print ("loading BERT setting")
	tokenizer = BertTokenizer.from_pretrained(model_path)
	train_inputs, train_labels, train_masks = data_prepare_BERT(train_file, lab2ind, tokenizer,content_col, label_col, max_seq_length)
	validation_inputs, validation_labels, validation_masks = data_prepare_BERT(dev_file, lab2ind, tokenizer, content_col, label_col,max_seq_length)
	# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
	model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(lab2ind))
	#--------------------------------------
	print ("[INFO] step (6) Create an iterator of data with torch DataLoader.")
#		  This helps save on memory during training because, unlike a for loop,\
#		  with an iterator the entire dataset does not need to be loaded into memory")
	train_data = TensorDataset(train_inputs, train_masks, train_labels)
	train_dataloader = DataLoader(train_data, batch_size=batch_size)
	#---------------------------
	validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
	validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
	#------------------------------------------
	print ("[INFO] step (7) run with parallel GPUs")
	if torch.cuda.is_available():
		if torch.cuda.device_count() == 1:
			print("Run", "with one GPU")
			model = model.to(device)
		else:
			n_gpu = torch.cuda.device_count()
			print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
			device_ids = GPUtil.getAvailable(limit = 4)
			torch.backends.cudnn.benchmark = True
			model = model.to(device)
			model = nn.DataParallel(model, device_ids=device_ids)
	else:
		print("Run", "with CPU")
		model = model
	#---------------------------------------------------
	print ("[INFO] step (8) set Parameters, schedules, and loss function")
	global max_grad_norm
	max_grad_norm = 1.0
	warmup_proportion = 0.1
	num_training_steps	= len(train_dataloader) * num_epochs
	num_warmup_steps = num_training_steps * warmup_proportion
	### In Transformers, optimizer and schedules are instantiated like this:
	# Note: AdamW is a class from the huggingface library
	# the 'W' stands for 'Weight Decay"
	optimizer = AdamW(model.parameters(), lr=lr_var, correct_bias=False)
	# schedules
	scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler
	# We use nn.MultiLabelSoftMarginLoss() as our loss function. 
	criterion = nn.MultiLabelSoftMarginLoss(weight=None, size_average=None, reduce=None, reduction='mean')
	#---------------------------------------------------
	print ("[INFO] step (9) start fine_tuning")
	for epoch in trange(num_epochs, desc="Epoch"):
		train_loss = train(model, train_dataloader, optimizer, scheduler, criterion)	  
		val_loss, val_acc, val_f1, val_recall, val_precision = evaluate(model, validation_dataloader, criterion)
# 		print (train_loss, val_acc)
		# Create checkpoint at end of each epoch
		if not os.path.exists(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/'): os.mkdir(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/')
		model.save_pretrained(ckpt_dir+ 'model_' + str(int(epoch + 1)) + '/')
		epoch_eval_results = {"epoch_num":int(epoch + 1),"train_loss":train_loss,
					  "val_acc":val_acc, "val_recall":val_recall, "val_precision":val_precision, "val_f1":val_f1,"lr":lr_var }
		print(epoch_eval_results)
		with open(report,"a") as fOut:
			fOut.write(json.dumps(epoch_eval_results)+"\n")
			fOut.flush()
		#------------------------------------
		report_df = pd.read_json(report, orient='records', lines=True)
		report_df.sort_values(by=[sortby],ascending=False, inplace=True)
		report_df.to_csv(sorted_report,sep="\t",index=False)
	return report_df

In [ ]:
report_df_mlsml = fine_tuning_MLSML(config_5)

[INFO] step (1) load train_test config file
[INFO] step (2) convert labels2index


loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file MARBERT_pytorch_verison/config.json


{'Mixed': 0, 'Negative': 1, 'Positive': 2}
./AJGT_MARBERT_labels-dict.json
[INFO] step (3) check checkpoint directory and report file
[INFO] step (4) load label to number dictionary
[INFO] train_file ./ar_rev_train.tsv
[INFO] dev_file ./ar_rev_test.tsv
[INFO] num_epochs 5
[INFO] model_path MARBERT_pytorch_verison
max_seq_length 128 batch_size 32
[INFO] step (5) Use defined funtion to extract tokanize data
loading BERT setting
Data size  (79999, 2)
The first sentence:
[CLS] رواية بسيطه عن مغترب تونسي يزور عائلته في اجازه قصيره ويلمس تغير البلاد والافكار الدينيه التي غزت العقول وحجم التناقض الكبير بين القول والفعل ..رواية جميله على عكس روايته السابقه روائح ماري كلير .. [SEP]
Tokenize the first sentence:
['[CLS]', 'رواية', 'بسيطه', 'عن', 'مغترب', 'تونسي', 'يزور', 'عايلته', 'في', 'اجازه', 'قصيره', 'ويلم', '##س', 'تغير', 'البلاد', 'والافكار', 'الدينيه', 'التي', 'غز', '##ت', 'العقول', 'وحجم', 'التناقض', 'الكبير', 'بين', 'القول', 'والفعل', '.', '.', 'رواية', 'جميله', 'على', 'عكس', 'رواي', '##

loading configuration file MARBERT_pytorch_verison/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file MARBERT_pytorch_verison/pytorch_

[INFO] step (6) Create an iterator of data with torch DataLoader.
[INFO] step (7) run with parallel GPUs
Run with one GPU


...

will return now


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_2/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_2/pytorch_model.bin
Epoch:  40%|████      | 2/5 [1:01:56<1:32:55, 1858.40s/it]

{'epoch_num': 2, 'train_loss': 0.5787095896303653, 'val_acc': 0.7362, 'val_recall': 0.7365670273223827, 'val_precision': 0.7367135054173034, 'val_f1': 0.7365874601617136, 'lr': 2e-06}


Epoch:  40%|████      | 2/5 [1:24:18<2:06:28, 2529.33s/it]


KeyboardInterrupt: ignored

In [ ]:
df3=pd.DataFrame(columns=['epoch_num',    'train_loss',    'val_acc',    'val_recall',    'val_precision',    'val_f1',    'lr'])
df3.loc[len(df3)]=[1,  0.728766883111,  0.7311, 0.7315776732115117,  0.7291082453475847, 0.7299571890529286,  2e-06]
df3.loc[len(df3)]=[2, 0.5787095896303653,  0.7362,  0.7365670273223827,  0.7367135054173034, 0.7365874601617136,  2e-06]


In [ ]:
print(df3)

splitting data into training and testing

In [ ]:
df = pd.read_csv("no_sw.tsv", sep='\t')
df_text_label = df[['cleaned_no_sw', 'label']]
print(df_text_label)
train, test = train_test_split(df_text_label, test_size=0.2, random_state=42, shuffle=True)
print(test)
train.to_csv("no_sw_train.tsv", sep='\t', index=False)
test.to_csv("no_sw_test.tsv", sep='\t', index=False)


                                           cleaned_no_sw     label
0      ممتاز نوعا . النظافة والموقع التجهيز الشاطيء ....  Positive
1      أحد أسباب نجاح الإمارات شخص الدولة يعشق تراها ...  Positive
2      هادفة وقوية . تنقلك صخب شوارع القاهرة الى هدوء...  Positive
3      خلصنا مبدئيا اللي مستوي انهار زي الفيل الازرق ...  Positive
4      سياسات جلوريا جزء لا يتجزأ دبي . فندق متكامل ا...  Positive
...                                                  ...       ...
99994  معرفش ليه كنت عاوزة أكملها مش ياباني البداية ا...  Negative
99995  لا يستحق ان يكون وكنت لانه سيئ . لا شي . لا يو...  Negative
99996  كتاب ضعيف استمتع . فى قصه سرد لحاله مشهد فكره قصه  Negative
99997  . محمد حسن عنوان فنان بالكلمات والنصف عندة دقي...  Negative
99998  ارجع اخرى . قربه البحر . المكان قديم ولا توجد ...  Negative

[99999 rows x 2 columns]
                                           cleaned_no_sw     label
26002  أقم الماشي . بصورة الغرفة جيدة شيء متوفر . عدم...  Positive
80420     تعجبني كباقي السلسلة متحمس

In [ ]:

config_no_sw={"task_name": "AJGT_MARBERT", #output directory name
             "data_dir": "./", #data directory
             "train_file": "no_sw_train.tsv", #train file path
             "dev_file": "no_sw_test.tsv", #dev file path or test file path
             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
             "epochs": 2, #number of epochs
             "content_col": "cleaned_no_sw", #text column
             "label_col": "label", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 128, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
report_df_stopwords = fine_tuning(config_no_sw)

[INFO] step (1) load train_test config file
[INFO] step (2) convert labels2index


loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file MARBERT_pytorch_verison/config.json


{'Mixed': 0, 'Negative': 1, 'Positive': 2}
./AJGT_MARBERT_labels-dict.json
[INFO] step (3) check checkpoint directory and report file
[INFO] step (4) load label to number dictionary
[INFO] train_file ./no_sw_train.tsv
[INFO] dev_file ./no_sw_test.tsv
[INFO] num_epochs 2
[INFO] model_path MARBERT_pytorch_verison
max_seq_length 128 batch_size 32
[INFO] step (5) Use defined funtion to extract tokanize data
loading BERT setting
Data size  (79983, 2)
The first sentence:
[CLS] رواية بسيطه مغتصب توني يزور عائلته اجازه قصيره يلمس تغير البلاد الافكار الدينيه غزت العقول وحجم التناقض الكبير القول بالفعل رواية جميله عكس روايته السابقه روائح ماري كلير [SEP]
Tokenize the first sentence:
['[CLS]', 'رواية', 'بسيطه', 'مغتصب', 'توني', 'يزور', 'عايلته', 'اجازه', 'قصيره', 'يلمس', 'تغير', 'البلاد', 'الافكار', 'الدينيه', 'غز', '##ت', 'العقول', 'وحجم', 'التناقض', 'الكبير', 'القول', 'بالفعل', 'رواية', 'جميله', 'عكس', 'رواي', '##ته', 'السابقه', 'روايح', 'ماري', 'كلي', '##ر', '[SEP]']
Index numbers of the first

loading configuration file MARBERT_pytorch_verison/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file MARBERT_pytorch_verison/pytorch_

[INFO] step (6) Create an iterator of data with torch DataLoader.
[INFO] step (7) run with parallel GPUs
Run with one GPU
[INFO] step (8) set Parameters, schedules, and loss function
[INFO] step (9) start fine_tuning


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

will return now
0.7258054706215858 0.7090709070907091


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_1/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_1/pytorch_model.bin
Epoch:  50%|█████     | 1/2 [31:20<31:20, 1880.12s/it]

will return now
0.6160594448089599 0.7107210721072107


Configuration saved in ./AJGT_MARBERT_bert_ckpt/model_2/config.json
Model weights saved in ./AJGT_MARBERT_bert_ckpt/model_2/pytorch_model.bin
Epoch: 100%|██████████| 2/2 [1:02:35<00:00, 1877.99s/it]


In [ ]:
report_df_stopwords[19:20]

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
20,2,0.616059,0.710721,0.711175,0.708451,0.708701,0.000002


# **Confusion Matrix Code**

In [ ]:
def conf_mat(model, iterator, criterion):
  model.eval()
  epoch_loss = 0
  all_pred=[]
  all_label = []
  with torch.no_grad():
    for i, batch in enumerate(iterator):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      input_ids, input_mask, labels = batch
      outputs = model(input_ids, input_mask, labels=labels)
      loss, logits = outputs[:2]
      # delete used variables to free GPU memory
      del batch, input_ids, input_mask
      if torch.cuda.device_count() == 1:
        epoch_loss += loss.cpu().item()
      else:
        epoch_loss += loss.sum().cpu().item()
      # identify the predicted class for each example in the batch
      probabilities, predicted = torch.max(logits.cpu().data, 1)
      # put all the true labels and predictions to two lists
      all_pred.extend(predicted)
      all_label.extend(labels.cpu())
    
  tp=0
  fp=0
  tn=0
  fn=0
  for p,l in all_pred, all_label:
    if p==l and p=="Positive":
      tp=tp+1
    if p==l and p=="Negative":
      tn=tn+1
    if p!=l and p=="Negative":
      fn=fn+1
    if p!=l and p=="Positive":
      fp=fp+1

  return tp,tn,fp,fn